FIR　EEG-ECoG 共通 EEG 予測 共通

1. FIR 0, IIR 1, LSTM 2
2. EEG-ECoG間でフィルター共通 0, バラバラ 1
3. 予測対象 EEG 0, ECoG 1
4. 予測の仕方 共通 0, NNをわける（フォルター特性が変わる） 1



In [1]:
# -*- coding: utf_8 -*-  
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn

import copy
import json
import pandas as pd
import numpy as np
import datetime
import math
import time

import chainer
from chainer import cuda, Function, gradient_check, utils, Variable
from chainer import optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from scipy import signal


import pickle
import sys
import random
import h5py

import fir_class

In [2]:
test=h5py.File('Ma/DeepANES.mat')

ECoG = np.array(test['ECoG'])
EEG = np.array(test['EEG'])
EEG = np.delete(EEG, 14, axis=1)
EEG = EEG.astype('float32')
ECoG = ECoG.astype('float32')

In [19]:
order = 5
time_range = 100

train_time = [0, time_range]
hamming = signal.hamming(train_time[1]-train_time[0])
#training ぶんだけにする
#EEG_t = np.zeros((EEG.shape[0]-order+1, EEG.shape[1], order))
#ECoG_t = np.zeros((ECoG.shape[0]-order+1, ECoG.shape[1], order))
EEG_t = np.zeros((train_time[1]-train_time[0], EEG.shape[1], order))
ECoG_t = np.zeros((train_time[1]-train_time[0], ECoG.shape[1], order))
EEG_tmp = EEG[train_time[0]:train_time[1], :] *np.array([hamming]*EEG.shape[1]).T
ECoG_tmp = ECoG[train_time[0]:train_time[1], :] *np.array([hamming]*ECoG.shape[1]).T
for t in range(order):
    for ch in range(EEG.shape[1]):
        #EEG_t[:,ch,t] = EEG[:EEG.shape[0]-order+1, ch] 
        EEG_t[t:,ch,t] = EEG_tmp[t:, ch] 
    for ch in range(ECoG.shape[1]):
        #ECoG_t[:,ch,t] = ECoG[:ECoG.shape[0]-order+1, ch] 
        ECoG_t[t:,ch,t] = ECoG_tmp[t:, ch] 
EEG_t = Variable((EEG_t[train_time[0]:train_time[1], :, :]).astype('float32'))
ECoG_t = Variable((ECoG_t[train_time[0]:train_time[1], :, :]).astype('float32'))

In [20]:
def calNN(timeSpacePerceptron, EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, loop=200):
    loss_history = np.zeros((loop, 1))
    #optimizer = optimizers.RMSprop()
    optimizer = optimizers.AdaDelta()
    optimizer.setup(timeSpacePerceptron)
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.05))
    #optimizer.add_hook(chainer.optimizer.Lasso(0.5))
    optimizer.add_hook(chainer.optimizer.GradientClipping(0.5))
    #timeSpacePerceptron.zerograds()
    for i in range(loop):
        timeSpacePerceptron.zerograds()
        x1, x2, EEG_filtered_prev, ECoG_filtered_prev = timeSpacePerceptron(EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, train=True)
        loss = F.mean_squared_error(x1, x2)
        loss.backward()
        optimizer.prepare()
        optimizer.update()
        loss_history[i] = loss.data
    return timeSpacePerceptron, loss_history, EEG_filtered_prev, ECoG_filtered_prev

In [21]:
def loop_calNN(EEG_t, ECoG_t, data_type, loop=200, loop_num=20):
    EEG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(EEG.shape[1])] #前の出力をいれるところ　(RNN)
    ECoG_filtered_prev = [Variable(np.zeros((time_range, 1)).astype('float32')) for j in range(ECoG.shape[1])] #前の出力をいれるところ
    saved_data = dict()
    outout = []
    out_loss_history = []
    for n in range(loop_num):
        out = []
        loss_history = np.zeros((loop, 1))
        timeSpacePerceptron = fir_class.__dict__["TimeSpacePerceptron{0}".format(data_type)](time_range)
        timeSpacePerceptron, loss_history, EEG_filtered_prev, ECoG_filtered_prev = calNN(timeSpacePerceptron, EEG_t, ECoG_t, EEG_filtered_prev, ECoG_filtered_prev, loop=loop)
        out_loss_history.append(loss_history)
        outout.append(timeSpacePerceptron)
    saved_data['loss_history'] = out_loss_history
    saved_data['NN'] = outout
    return saved_data

In [22]:
saved_data = dict()
loop = 50
loop_num = 1
st = time.time()

# channle をわけない
data_type ='0000'
saved_data[data_type] =  loop_calNN(EEG_t, ECoG_t, data_type, loop=loop, loop_num=loop_num)
print(time.time()-st)

data_type ='0100'
saved_data[data_type] =  loop_calNN(EEG_t, ECoG_t, data_type, loop=loop, loop_num=loop_num)
print(time.time()-st)

data_type ='0010'
saved_data[data_type] =  loop_calNN(EEG_t, ECoG_t, data_type, loop=loop, loop_num=loop_num)
print(time.time()-st)

data_type ='0110'
saved_data[data_type] =  loop_calNN(EEG_t, ECoG_t, data_type, loop=loop, loop_num=loop_num)
print(time.time()-st)


12.168619155883789
24.177961111068726
35.94694805145264
47.4869270324707


In [23]:
# channle をわける
# EEG
data_type ='0001'
tmp_data = []
for ch in range(EEG.shape[1]):
    tmp_data.append(loop_calNN(EEG_t[:, ch, :], ECoG_t, data_type, loop=loop, loop_num=loop_num))
saved_data[data_type] =  tmp_data
print(time.time()-st)

data_type ='0101'
tmp_data = []
for ch in range(EEG.shape[1]):
    tmp_data.append(loop_calNN(EEG_t[:, ch, :], ECoG_t, data_type, loop=loop, loop_num=loop_num))
saved_data[data_type] =  tmp_data
print(time.time()-st)

# ECoG
data_type ='0011'
tmp_data = []
for ch in range(ECoG.shape[1]):
    tmp_data.append(loop_calNN(EEG_t, ECoG_t[:, ch, :], data_type, loop=loop, loop_num=loop_num))
saved_data[data_type] =  tmp_data
print(time.time()-st)

data_type ='0111'
tmp_data = []
for ch in range(ECoG.shape[1]):
    tmp_data.append(loop_calNN(EEG_t, ECoG_t[:, ch, :], data_type, loop=loop, loop_num=loop_num))
saved_data[data_type] =  tmp_data
print(time.time()-st)


226.63573098182678
404.01942706108093
576.5715849399567
759.396605014801


/Users/vv000844/.pyenvi/versions/anaconda3-2.5.0/lib/python3.5/site-packages/chainer/optimizer.py:520: RuntimeWarning: invalid value encountered in multiply
  grad *= rate


In [ ]:
pickle.dump(saved_data, open('result_FIR_{0}{1}.pkl'.format(loop, loop_num), 'wb'))

In [ ]:
#train_time[1]-train_time[0]

In [ ]:
#eeg_reconstruct, eeg_filtered = outout[2][99](EEG_t, ECoG_t, train=False)

In [ ]:
#for i in range(16):
#    print(np.corrcoef(eeg_reconstruct.data[:,i], eeg_filtered.data[:,i])[0, 1])